In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import matplotlib

from oracles import LogReg

from methods import Standard_Newton, Newton_Star
from methods import NL1, NL1_Bernoulli
from methods import NL2, NL2_Bernoulli
from methods import CNL, CNL_Bernoulli
from methods import DINGO, dcgd
from methods import diana, adiana

from utils import read_data
from utils import loss_logistic, grad, random_k, positive_part
from utils import default_dataset_parameters
from utils import topK_vectors, biased_rounding, randomK_vectors
from utils import Low_Rank, PowerSgdCompression, TopK
from utils import pos_projection, semidef_projection
from utils import random_spars_matrix, rand_dith

from easydict import EasyDict

In [ ]:
data_name = 'a2a'
dataset_path = './Datasets/{}.txt'.format(data_name)


# regularization parameter
lmb = 1e-3

# number of nodes, size of local data, and dimension of the problem
# according to the paper
N = default_dataset_parameters[data_name]['N']# size of the whole data set
n = default_dataset_parameters[data_name]['n']# number of nodes
m = default_dataset_parameters[data_name]['m']# size of local data set
d = default_dataset_parameters[data_name]['d']# dimension of the problem

In [ ]:
A, b = read_data(dataset_path=dataset_path, 
                 N=N, n=n, m=m, d=d, lmb=lmb,
                labels=['+1', '-1'])

# labels are {+1, -1} for a2a, a7a, a9a, w8a datasets
# labels are {0, 1} for phishing

In [ ]:
# set the problem 
logreg = LogReg(A=A, b=b, reg_coef=lmb, n=n, m=m)

# find the solution using Newton's method starting from zeros for 20 iterations
Newton = Standard_Newton(logreg)
Newton.find_optimum(x0=np.zeros(d), n_steps=20,verbose=True)

In [ ]:
# set optimum and optimal function value
x_opt = logreg.get_optimum()
f_opt = logreg.function_value(x_opt)

# Newton-Learn 1 (NL1)

***input:***
- `x` - initial model weightsn
- `H` - list of vectors h_i^0
- `max_iter` - maximum number of iterations of the method
- `k` - the parameter of Rand-K compression operator
- `eta` - stepsize for update of vectors h_i's
  - if `eta` is None, then `eta` is set as k/m)
- `tol` - desired tolerance of the solution
- `init_cost` 
  - if True, then the communication cost of initalization is inclued
- `verbose` 
  - if True, then function values in each iteration are printed

return:
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.1

# define the method
nl1 = NL1(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = nl1.method(x=x, H=H, init_cost=False, k=1, eta=1/m,
                        tol=1e-15, max_iter=20, verbose=True)

## Newton-Learn 1 (NL1) with induced Bernoulli compressor 

All parameters are the same as above, but one more parameter `p` is included as a parameter of Bernoulli parameter. Note that here communicated bits per node computed without paying attention on Bernoulli compressor. If you want to make fair comparison, you need to calculate bits sended by all "active" nodes (multiply by $pn$).  

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.1

# define the method
nl1_b = NL1_Bernoulli(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = nl1_b.method(x=x, p=1/2, H=H, init_cost=False, k=1, eta=1/m,
                          tol=1e-15, max_iter=20, verbose=True)

# Newton-Learn 2 (NL2)

***input:***
- `x` - initial model weightsn
- `H` - list of vectors h_i^0
- `gamma` - parameter of NL2
- `max_iter` - maximum number of iterations of the method
- `k` - the parameter of Rand-K compression operator
- `eta` - stepsize for update of vectors h_i's
  - if `eta` is None, then `eta` is set as k/m)
- `tol` - desired tolerance of the solution
- `init_cost` 
  - if True, then the communication cost of initalization is inclued
- `verbose` 
  - if True, then function values in each iteration are printed

return:
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.1

# define the method
nl2 = NL2(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = nl2.method(x=x, H=H, gamma=0.5, init_cost=False, k=1, eta=1/m,
                        tol=1e-15, max_iter=20, verbose=True)

## Newton-Learn 2 (NL2) with induced Bernoulli compressor 

All parameters are the same as above, but one more parameter `p` is included as a parameter of Bernoulli parameter. Note that here communicated bits per node computed without paying attention on Bernoulli compressor. If you want to make fair comparison, you need to calculate bits sended by all "active" nodes (multiply by $pn$).  

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.1

# define the method
nl2_b = NL2_Bernoulli(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = nl2_b.method(x=x, H=H, gamma=0.5, p=1/2, init_cost=False, k=1, eta=1/m,
                        tol=1e-15, max_iter=20, verbose=True)

# Cubic-Newton-Learn (CNL)

***input:***
- `x` - initial model weightsn
- `H` - list of vectors h_i^0
- `nu`, `gamma` - parameteres of CNL
- `max_iter` - maximum number of iterations of the method
- `k` - the parameter of Rand-K compression operator
- `eta` - stepsize for update of vectors h_i's
  - if `eta` is None, then `eta` is set as k/m)
- `tol` - desired tolerance of the solution
- `init_cost` 
  - if True, then the communication cost of initalization is inclued
- `verbose` 
  - if True, then function values in each iteration are printed

return:
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point 
x = x_opt + np.ones(d)*1

# define the method
cnl = CNL(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(np.zeros(m))
    #H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = cnl.method(x=x, H=H, nu=1/8, gamma=0.5, init_cost=False, k=1, eta=1/m,
                        tol=1e-15, max_iter=20, verbose=True)

## Cubic-Newton-Leanr (CNL) with induced Bernoulli compressor

All parameters are the same as above, but one more parameter `p` is included as a parameter of Bernoulli parameter. Note that here communicated bits per node computed without paying attention on Bernoulli compressor. If you want to make fair comparison, you need to calculate bits sended by all "active" nodes (multiply by $pn$).

In [ ]:
# initial point 
x = x_opt + np.ones(d)*1

# define the method
cnl_b = CNL_Bernoulli(logreg)

# define vectors h_i's as second derivatives at the initial point
H = []
for i in range(n):
    H.append(np.zeros(m))
    #H.append(logreg.alphas(x,i))

# run the method
fv, bi, it = cnl_b.method(x=x, H=H, nu=1/8, gamma=0.5, init_cost=False, k=1, eta=1/m,
                        tol=1e-15, max_iter=20, verbose=True)

# Newton-Star

***input:***
- `x` - initial model weightsn
- `init_cost` - if True, then the communication cost of initalization is inclued
- `tol` - desired tolerance of the solution
- `max_iter` - maximum number of iterations
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server
- `iterates` - numpy array containing distances from current point to the solution

In [ ]:
# initial point 
x = x_opt + np.ones(d)*0.2

# define the method
n0 = Newton_Star(logreg)

# run the method
fv, bi, it = n0.method(x0=x, init_cost=False,
                      tol=1e-15, max_iter=100, verbose=True)

# DINGO

***input:***
- `x` - initial point
- `max_iter` - maximum iterations of the method
- `tol` - desired tolerance of the solution
- `phi`, `theta`, `rho` - parameters of DINGO
- `verbose` - if True, then function values in each iteration are printed

***return:***
- `func_value` - numpy array containing function value in each iteration of the method
- `bits` - numpy array containing transmitted bits by one node to the server

In [ ]:
# initial point
x = x_opt + np.ones(d)*0.1

# define method
dingo = DINGO(logreg)

# run the method
fv, bi = dingo.method(x=x, phi=1e-6, theta=1e-4, 
                      rho=1e-4, tol=1e-15, 
                      max_iter=20, verbose=True)

# DCGD, ADIANA and DIANA

In [ ]:
# eta, alpha, theta_1, theta_2, gamma are parameters of ADIANA 
class args(EasyDict):
    def __init__(self, data_name, T, node, L, lamda, eta=0.05,
                 alpha=0.5, theta_1=0.25, theta_2=0.5, gamma=0.5,
                 beta=0.95,
                 prob=1,
                 comp_method='no_comp',
                 r=None, s=None, s_level=10):
        super().__init__()
        self.data_name = data_name
        # T - maximum number of iterations
        self.T = T
        # number of nodes
        self.node = node
        # parameters of ADIANA
        self.eta = eta
        self.alpha = alpha
        self.theta_1 = theta_1
        self.theta_2 = theta_2
        self.gamma = gamma
        self.beta = beta
        self.prob = prob
        # Lipshitz constant of gradients
        self.L = L
        # regularization parameter
        self.lamda = lamda
        # name of compression operator
        self.comp_method = comp_method
        # parameter of random sparsification
        # compression operator
        self.r = r
        # parameter of randim dithering
        # compression operator
        self.s = s
    
# find Lipshitz constant of gradients
H = np.dot(A.T,A)/N
temp = np.linalg.eigvalsh(H)
L = np.abs(temp[-1])/4

# define maximum number of iterations
max_iter = 200

# set class containing all parameters
arg = args(data_name, max_iter, n, L, lmb)

# set parameters of compression operators
arg.r = d/4
arg.s = np.sqrt(d)

# set compression operator
comp_methods = [
    'rand_dithering',
    'rand_sparse',
    'natural_comp'
]

# put compression operator to arg's
arg.comp_method = comp_methods[0]

# initial point
x = x_opt + np.ones(d)*0.1

In [ ]:
loss_dcgd, com_bits_dcgd = dcgd(A, b, x, arg, f_opt=f_opt, tol=1e-3)

In [ ]:
# run ADIANA
loss_adiana, com_bits_adiana = adiana(A, b, x, arg, f_opt=f_opt, tol=1e-15)

# method prints function values each 1000 iterations

In [ ]:
# run ADIANA

loss_diana, com_bits_diana = diana(A, b, x, arg, f_opt=f_opt, tol=1e-15)
# method prints function values each 1000 iterations